In [1]:
import multiprocessing
import pandas as pd
import os
import numpy as np
import json
import math
import threading
import gc
import time

In [2]:
datasets = os.path.join(os.getcwd(),'Datasets')

movies_path = os.path.join(datasets,'movies_dataset')

user_based_dict_path = os.path.join(movies_path,'utility_matrix_user_based.txt')
item_based_dict_path = os.path.join(movies_path,'utility_matrix_item_based.txt')
metadata_path = os.path.join(movies_path,'movies_metadata.txt')
cliques_path = os.path.join(movies_path, 'cliques_items.txt')


CORES = multiprocessing.cpu_count()
cliques = {}

In [57]:
# don't know why this is here, remove if not needed

new_cliques = {}
items_keys = list(cliques.keys())
for item in items_keys:
    new_list = []
    for sub_array in cliques[item]:
        new_list.append(list(sub_array))
        
     
    new_cliques[item] = new_list

In [48]:
with open(cliques_path,'r') as fr:
    new_cliques = json.load(fr)

In [49]:
new_cliques['2654.0'][0:5]

[[2651.0, 0.24268948974477672],
 [2650.0, 0.2201435629404818],
 [2638.0, 0.175415486335374],
 [2644.0, 0.17410660068041184],
 [3472.0, 0.17328611957624648]]

In [50]:
for item in new_cliques.keys():
    clique_item = new_cliques[item]
    clique_item_np = []
    for pair in clique_item:
        clique_item_np.append(np.array(pair))
    clique_item_np = np.array(clique_item_np)
    
    item_float = float(item)
    cliques[item_float] = clique_item_np

In [51]:
cliques[2654][0:5]

array([[2.65100000e+03, 2.42689490e-01],
       [2.65000000e+03, 2.20143563e-01],
       [2.63800000e+03, 1.75415486e-01],
       [2.64400000e+03, 1.74106601e-01],
       [3.47200000e+03, 1.73286120e-01]])

In [52]:
len(list(cliques.keys()))

3797

In [8]:
# load the metadata file, with topic informations ecc
with open(metadata_path,'r') as fr:
    movies_metadata = json.load(fr)


def convert_metadata_to_int(movies_metadata):
    movies_metadata_int = {}

    for key1 in movies_metadata.keys():
        key11 = int(key1)

        movies_metadata_int[key11] = movies_metadata[key1]
    
    return movies_metadata_int

movies_metadata = convert_metadata_to_int(movies_metadata)

In [9]:
# load item based dictionary dataset
with open(user_based_dict_path,'r') as fr:
    user_based_utility_matrix = json.load(fr)

In [10]:
def convert_ratings_to_float(user_based_utility):
    user_based_correct = {}

    for key1 in user_based_utility.keys():
        for key2 in user_based_utility[key1].keys():

            key11 = int(float(key1))
            key22 = int(float(key2))

            if(user_based_correct.get(key11) == None):
                user_based_correct[key11] = {}
            
            user_based_correct[key11][key22] = float(user_based_utility[key1][key2])
    
    return user_based_correct

user_based_utility_matrix = convert_ratings_to_float(user_based_utility_matrix)

gc.collect()

0

In [11]:
removed = []

def remove_items_not_in_metadata(user_based_utility, movies_metadata):
    for key1 in user_based_utility.keys():
        dict_1 = user_based_utility[key1]
        keys = list(dict_1.keys()).copy()
        for idx in range(len(keys)):
            key2 = keys[idx]
            if movies_metadata.get(key2) == None:
                print('item {} has no metadata, eliminating'.format(key2))
                removed.append(key2)
                del dict_1[key2]
    return user_based_utility, removed

user_based_utility_matrix, removed = remove_items_not_in_metadata(user_based_utility_matrix, movies_metadata)
removed_unique = np.unique(removed)
removed_unique

1 has no metadata, eliminating
item 52281 has no metadata, eliminating
item 27611 has no metadata, eliminating
item 150548 has no metadata, eliminating
item 27611 has no metadata, eliminating
item 52281 has no metadata, eliminating
item 7669 has no metadata, eliminating
item 27611 has no metadata, eliminating
item 52281 has no metadata, eliminating
item 150548 has no metadata, eliminating
item 155968 has no metadata, eliminating
item 720 has no metadata, eliminating
item 52281 has no metadata, eliminating
item 720 has no metadata, eliminating
item 52281 has no metadata, eliminating
item 85780 has no metadata, eliminating
item 27611 has no metadata, eliminating
item 720 has no metadata, eliminating
item 62336 has no metadata, eliminating
item 7669 has no metadata, eliminating
item 62336 has no metadata, eliminating
item 26693 has no metadata, eliminating
item 4207 has no metadata, eliminating
item 720 has no metadata, eliminating
item 27611 has no metadata, eliminating
item 52281 has no

array([   142,    720,    730,    769,    770,    791,    821,   1107,
         1122,   1133,   1142,   1166,   1316,   1421,   1434,   1709,
         2258,   2270,   2851,   4051,   4207,   4568,   5069,   5209,
         5738,   6955,   7669,  26379,  26587,  26649,  26693,  27049,
        27611,  27708,  27724,  30991,  31193,  32352,  32600,  38198,
        40697,  42987,  52281,  55207,  62336,  69849,  70828,  73370,
        73759,  77328,  77330,  77359,  77854,  79299,  81731,  85780,
        87073,  87308,  90647,  90945,  93988,  96062,  96075,  99766,
       100044, 100058, 100089, 100450, 100538, 103277, 104155, 106334,
       106642, 107718, 107780, 108727, 108977, 118392, 119565, 122926,
       126106, 139195, 143351, 144050, 147250, 150548, 151763, 152284,
       155968, 163921, 167570, 169906, 170745, 171749, 173535])

In [12]:
# functions to operate division in test/train dictionaries


def divide_dataset(dataset):
    folds = [{},{},{},{}]

    for key1 in dataset.keys():
        for key2 in dataset[key1].keys():
            rand_index = np.random.randint(4)
            
            tmp_dict = folds[rand_index]

            if(tmp_dict.get(key1) == None):
                tmp_dict[key1] = {}
            
            tmp_dict[key1][key2] = dataset[key1][key2]
    
    return folds

def merge_dicts(dicts):
    ret = {}

    for i in range(len(dicts)):
        tmp_dict = dicts[i]
        for key1 in tmp_dict.keys():
            for key2 in tmp_dict[key1].keys():

                if(ret.get(key1)==None):
                    ret[key1] = {}
                ret[key1][key2] = tmp_dict[key1][key2]

    return ret


# divide the dataset in train/test set
np.random.seed(10)
folds = divide_dataset(user_based_utility_matrix)

train_dict = merge_dicts([fold for fold in folds[:3]])
test_dict = folds[3] 

#try to clear memory usage
del user_based_utility_matrix
del folds
gc.collect()

0

In [13]:
def invert_train_dictionary(train_dict):
    inverted_train_dict = {}
    for user in train_dict.keys():
        user_dict = train_dict[user]
        for item in user_dict.keys():
            if inverted_train_dict.get(item) == None:
                inverted_train_dict[item] = {}
            inverted_train_dict[item][user] = train_dict[user][item]
    return inverted_train_dict

inverted_train_dict = invert_train_dictionary(train_dict)

In [14]:
# function used in computing similarity measure between two users 


"""
    Calculates the average value of the 
    valeues in a

    a -> dictionary of non zero values 
"""
def avg(a):
    i=0
    tot = 0
    for k in a.keys():
        i +=1
        tot += a.get(k)
    
    return tot/i

"""
    Calculates a new dictionary that for each
    non null value of a has that values - const

    returns a - const

    a -> dictioary of non zero values
"""
def scale(a,const):
    ret = {}
    
    for k in a.keys():
        ret[k] = a.get(k) - const
    
    return ret

"""
    calculates the norm of a dictionary

    a -> dictioary of non zero values
"""
def norm (a):
    tot = 0
    for k in a.keys():
        tot += pow(a.get(k),2) 

    return math.sqrt(tot)

"""
    computes the inner product of two dictionaries

    a -> dictioary of non zero values
    b -> dictioary of non zero values

"""
def inner_product(a,b):
    tot = 0
    for k in a.keys():
        b_tmp = b.get(k)
        if(b_tmp != None):
            tot += b_tmp * a.get(k) 

    return tot

"""
    Calculates the correlation coefficent
        between two vectors
    
    a, b dictionaries of non zero values

    esplicit -> boolean value that  defines if we
    need to scale or not cause in the case of implicit
    we do not need to scale
"""
def compute_correlation_coefficent(a,b):

    avg_a = avg(a)
    avg_b = avg(b)

    a_scaled = scale(a, avg_a)
    b_scaled = scale(b, avg_b)

    a_scaled_norm = norm(a_scaled)
    b_scaled_norm = norm(b_scaled)

    if(a_scaled_norm == 0 or b_scaled_norm == 0):
        # print('One of the two vectors has 0 norm, returning 0.')
        return 0 

    sim = inner_product(a_scaled,b_scaled)/(a_scaled_norm*b_scaled_norm)

    return sim

def compute_similarities(item_ids, utility_matrix, item_dict):
    similarities = []
    for item in item_ids:
        tmp_item_dict = utility_matrix[item]
        similarity = compute_correlation_coefficent(item_dict, tmp_item_dict)
        similarities.append([item,similarity])
    
    return similarities


# definition of threads that will compute similarities for cliques generation
class ComputeCorrelationCoefficentThread (threading.Thread):
   """
      user_dict -> dictionary of the user: {itemID:rating}
      user_ids -> set of users IDs
      utility_matrix -> dataset
   """
   def __init__(self, item_dict, item_ids, utility_matrix, name = None):
      threading.Thread.__init__(self)
      self.name = name
      self.item_dict = item_dict
      self.item_ids = item_ids
      self.utility_matrix = utility_matrix
      self.result = None
   
   def run(self):
      if self.name == None:
         raise Exception('Something bad happened, thread has no name')
      #print('Thread {} started'.format(self.name))
      self.result = compute_similarities(self.item_ids, self.utility_matrix, self.item_dict)
      

   # vector of similarities --> [userID, sim_score]
   def join(self):
        threading.Thread.join(self)
        if self.result is not None:
            return self.result
        else:
            print('Error using threads, result is None')

In [15]:
# partition a list of unique items (data) into num_folds sublists

def make_partitions(num_folds, data):
    data_size = len(data)
    fold_size = data_size // num_folds
    partitions = []
    for idx in range(num_folds-1):
        partitions.append(data[idx*fold_size:(idx+1)*fold_size])
    partitions.append(data[idx*fold_size:])
    return partitions


"""
    Returns the clique of the user as np.array

    user -> userID
    utility_matrix -> {user:{item:rating}}
    clique_size -> size of the clique of the user
"""
def compute_clique(item, utility_matrix, clique_size):

    if type(cliques.get(item)) is np.ndarray:
        print('\nclique already computed, use existing one')
        return cliques[item]

    unique_items = list(utility_matrix.keys())

    # remove the item itself from the items to compare with
    unique_items.remove(item)
    item_dict = utility_matrix[item]
    
    partitions = make_partitions(CORES,unique_items)        

    threads = []
    for idx in range(CORES):
        name = "Thread-{}".format(idx)
        thread = ComputeCorrelationCoefficentThread(name = name, 
                                                item_dict = item_dict, 
                                                item_ids = partitions[idx], 
                                                utility_matrix = utility_matrix)
        threads.append(thread)

    
    [thread.start() for thread in threads]    
    print('\n======= {} threads started ======='.format(CORES))

    results = [thread.join() for thread in threads]
    print('\n======= all threads joined =======')
    
    similarities = []
    for elem in results:
        for similarity in elem:
            similarities.append(similarity)

    similarities = np.array(similarities)
    clique = similarities[np.argsort(similarities[:,1])[::-1]][:clique_size]

    cliques[item] = clique

    return clique

In [16]:
"""
    returns the predicted rating given an item and an user

    item -> itemID
    user -> userID
    clique -> list of items that are similar to itemID
"""


def predict(item, user, clique, train_dictionary):
    numerator = 0
    denominator = 0
    for elem in clique:
        neighbor = elem[0]
        similarity = elem[1]

        neigh_dict = train_dictionary[neighbor]
        rating = neigh_dict.get(user)
        
        if  rating == None:
            continue

        numerator += rating*similarity
        denominator += similarity
    
    if denominator == 0:
        print('denominator is 0, no item in the clique has been rated by this user')
        return -1
    
    return numerator/denominator

In [17]:
# randomly select a subset of k users in test_dict for which we will score the recommender system 
# test items must have at least k ratings to test (so that, after removing the ones that we cannot predict (if no item in the clique has been rated by the same user) we should have 10 ratings to rank)

def select_test_users(train_dict, test_dict, movies_metadata, test_size, min_ratings_per_user):
    np.random.seed(7)

    selected_test_users = []

    all_test_users = list(test_dict.keys())
    print('\nThe number of users in test set is {}'.format(len(all_test_users), flush=True))
    print('\nWe are taking into consideration only users with more than {} ratings'.format(min_ratings_per_user))
    print('\nThe subset of the test set taken into consideration is of size {}'.format(test_size))


    i=1
    while i<=test_size:
        #print('i is {}'.format(i), flush=True)
        idx = np.random.randint(len(all_test_users))
        user = all_test_users[idx]

        '''
        # don't care if user is not in train set, since I need only the items it rated to be there
        if user not in all_train_users:
            print("User is not in train set, can't compute clique")
            continue
        '''

        # check if item has at least k ratings
        num_user_ratings = len(list(test_dict[user].keys()))
        if num_user_ratings < min_ratings_per_user:
            #print('\nUser has {} ratings, skipping'.format(num_user_ratings), flush=True)
            continue      

        # check if we already considered this user
        if user in selected_test_users:
            #print('user already considered', flush=True)
            continue

        # add user to list of selected users
        selected_test_users.append(user)
        i+=1

    return selected_test_users
    

In [18]:
lista = select_test_users(0, test_dict, movies_metadata, 100, 30)
lista


The number of users in test set is 232351

We are taking into consideration only users with more than 30 ratings

The subset of the test set taken into consideration is of size 100


[221093,
 62448,
 79554,
 109497,
 135635,
 143955,
 265660,
 218583,
 267230,
 202638,
 216529,
 46210,
 226298,
 228606,
 137233,
 169004,
 140345,
 60227,
 143341,
 24561,
 256238,
 217266,
 129628,
 90360,
 219349,
 268392,
 215252,
 59826,
 21908,
 26108,
 204871,
 170185,
 23275,
 223988,
 176898,
 2751,
 226587,
 2645,
 260048,
 77064,
 235436,
 32198,
 36489,
 242604,
 174028,
 169300,
 88160,
 115184,
 138889,
 210485,
 14009,
 209279,
 264313,
 37187,
 219367,
 266783,
 198723,
 189363,
 173996,
 149719,
 56258,
 147857,
 94532,
 203536,
 69304,
 191542,
 249350,
 167818,
 39632,
 219141,
 200020,
 39246,
 23811,
 56687,
 13710,
 123037,
 158085,
 174001,
 2203,
 261933,
 29855,
 99052,
 766,
 97318,
 187351,
 152593,
 157150,
 235018,
 195456,
 168246,
 250906,
 100327,
 69549,
 168388,
 215768,
 152170,
 166280,
 262458,
 74649,
 31591]

In [19]:
# list value is computed by taking the true rating of each element returned multyplied with a factor that depends on the position at which the element is in the ranked recommendation predicted

def compute_list_value(predicted_ranking, true_ranking):
    list_value = 0
    for index, row in predicted_ranking.iterrows():
        rating = true_ranking[true_ranking['item'] == row['item']]['true_rating'].values[0]
        item_score = (1/(np.log(index+np.e)))*rating 
        list_value += item_score

        # print(index, rating)
    return list_value

In [20]:
# definition of jacccard coefficient score and dissimilarity measure


default_similarity = 0.1
def jaccard_score(list_1, list_2):
    intersection = []
    union = []
    for elem in list_1:
        if elem not in intersection and elem in list_2:
            intersection.append(elem)
        union.append(elem)
    for elem in list_2:
        if elem not in union:
            union.append(elem)


    try:
        score = len(intersection)/len(union)
    except:
        print('division by zero, assuming a similarity of {}'.format(default_similarity))
        return default_similarity
    return score

def dissimilarity_measure(id_1, id_2, movies_metadata):
    metadata_1 = movies_metadata[id_1]
    metadata_2 = movies_metadata[id_2]

    #spoken_language_1 = metadata_1['spoken_language']
    #spoken_language_2 = metadata_2['spoken_language']

    original_language_1 = [metadata_1['original_language']]
    original_language_2 = [metadata_2['original_language']]

    production_companies_1 = metadata_1['production_companies']
    production_companies_2 = metadata_2['production_companies']

    production_countries_1 = metadata_1['production_countries']
    production_countries_2 = metadata_2['production_countries']

    genres_1 = metadata_1['genres']
    genres_2 = metadata_2['genres']

    #spoken_language_score = jaccard_score(spoken_language_1, spoken_language_2)
    original_language_score = jaccard_score(original_language_1, original_language_2)
    production_companies_score = jaccard_score(production_companies_1, production_companies_2)
    production_countries_score = jaccard_score(production_countries_1, production_countries_2)
    genres_score = jaccard_score(genres_1, genres_2)    

    scores = np.array([original_language_score, production_companies_score, production_countries_score, genres_score])

    weights = np.array([0.15, 0.15, 0.1, 0.6])

    similarity = np.dot(weights, scores)
    return 1-similarity

In [21]:
def intra_list_similarity(l,movies_metadata):
    similarity = 0
    n_items = len(l)

    for i in range(n_items - 1):
        for j in range(i + 1, n_items):
            (item1, item2) = (l[i], l[j])
            similarity += 1-dissimilarity_measure(item1, item2, movies_metadata)

    return similarity

In [22]:
def differentiation_algorithm(old_list, movies_metadata, diversification_factor):
    new_list = old_list[:1] # The first element remains always the same
    old_ranking = [list((rank, old_list[rank - 1], 0)) for rank in range(1, len(old_list) + 1)] # List of triples (rank(starting from 1), item, dissimilarity_actual_value)
    new_item = new_list[0]
    

    for i in range(1, 10):
        old_ranking = list(filter(lambda x: x[1] != new_item, old_ranking)) # Remove the element passed in the previous iteration to the new list from the old ranking
        dissimilarities = []
        
        for index in range(len(old_ranking)):
            triple = old_ranking[index]
            old_ranking[index][2] += dissimilarity_measure(int(triple[1]), int(new_item), movies_metadata) # Update the dissimilarity value considering the last added item (avg)
            dissimilarities.append(old_ranking[index])
        
        dissimilarities.sort(reverse=True, key=lambda tup: tup[2]) # Sort in decreasing order according to the dissimilarity value
        dissimilarity_rank = 0
        min_rank = len(old_list) + 1 + len(dissimilarities)
        
        for triple in dissimilarities:
            new_rank = triple[0] * (1 - diversification_factor) + dissimilarity_rank * diversification_factor # We consider the rank for both the old list and the dissimilarity sorted list
            if new_rank < min_rank:
                new_item = triple[1]
                min_rank = new_rank
            dissimilarity_rank += 1
        
        new_list.append(new_item)
    
    return new_list

In [23]:
# select optimal ranking, predicted ranking, and optimal top 10, then score the result using list_value metric
def score_predictions(predictions, movies_metadata, diversification_factor, k):
    optimal_ranking = predictions[['item', 'true_rating']].sort_values(by=['true_rating'], ascending=False).reset_index(drop=True)

    predicted = predictions[['item', 'predicted_rating']].sort_values(by=['predicted_rating'], ascending=False).reset_index(drop=True)

    if predicted.shape[0] > k:
        predicted = predicted[:k]

    assert(predicted.shape[0] <= 50, 'predicted still contains more than 50 elements, something wrong happened')    

    if diversification_factor != 0:
        # print('diversification factor != 0, start differentiating')
        predicted_top_10 = differentiation_algorithm(list(predicted['item'].values), movies_metadata, diversification_factor)
        mask = [elem in predicted_top_10 for elem in predicted['item'].values]
        predicted_top_10 = predicted[mask].reset_index(drop=True)
    else:
        predicted_top_10 = predicted[:10]
    optimal_top_10 = optimal_ranking[:10]

    # compute similarity score of the original list and the diversified list
    sim_original = intra_list_similarity(list(predicted['item'][:10].values), movies_metadata)
    sim_diversified = intra_list_similarity(list(predicted_top_10['item'].values), movies_metadata)

    # compute list value both for the optimal ranking and the predicted one
    list_value = compute_list_value(predicted_top_10, optimal_ranking)
    best_list_value = compute_list_value(optimal_top_10, optimal_ranking)
    
    # the metric is MAE, considering as error the difference of the list values of the optimal top 10 list and the predicted one
    difference = best_list_value - list_value
    print('Optimal ranking value: {}\tpredicted ranking value: {}\t difference: {}'.format(best_list_value, list_value,difference))

    return difference, sim_original, sim_diversified

In [59]:
def perform_test(train_dict, test_dict, movies_metadata, clique_size = 50, test_size = 5, min_ratings_per_user = 20, differentiation_factor = 0, k=50):

    print('\nClique size is {}'.format(clique_size))

    selected_test_users = select_test_users(train_dict, test_dict, movies_metadata, test_size, min_ratings_per_user)
    assert(len(selected_test_users) == test_size, "Sizes don't match: something is wrong with selecting test users!")

    MAE = 0
    mean_original_similarity = 0
    mean_diversified_similarity = 0
    i=1
    for user in selected_test_users:

        print('user number {}'.format(i))
        i+=1
        
        test_items = list(test_dict[user].keys())
        
        # if an item is not in train_set too, I have to remove its id from the list of considered
        all_train_items = list(train_dict.keys())

        test_items_cliques = {}
        for item in test_items:
            
            # remove an item that is not present in train items
            if item not in all_train_items:
                print('\nItem is not in train dictionary, I have to skip this one')
                continue

            clique = compute_clique(item, train_dict, clique_size)

            test_items_cliques[item] = clique

        test_items = list(test_items_cliques.keys())

        # check if predict has this structure
        predictions_and_truth = pd.DataFrame( np.array( [[item, predict(item, user, test_items_cliques[item], train_dict), test_dict[user][item]] for item in test_items] ),columns=['item','predicted_rating','true_rating'] )
        predictions_and_truth = predictions_and_truth[predictions_and_truth.predicted_rating != -1]

        difference, sim_original, sim_diversified = score_predictions(predictions_and_truth, movies_metadata, differentiation_factor,k)
        assert(difference >= 0, 'Error: list value greater than best list value')
        
        MAE += difference
        mean_original_similarity += sim_original
        mean_diversified_similarity += sim_diversified
        
        print('\nintra-list similarity score of original list is {}, while the diversified list scored {}'.format(sim_original, sim_diversified))
        

    MAE = MAE/test_size
    mean_original_similarity = mean_original_similarity/test_size
    mean_diversified_similarity = mean_diversified_similarity/test_size
    
    print('\n======= TEST TERMINATED ======= ')
    print('Mean absolute error is {}\nMean intra-similarity score for non-diversified predictions is {}\nMean intra-similarity score for diversified predictions is {}\nResults have been computed on a subset of the test dictionary of {} users'.format(MAE, mean_original_similarity, mean_diversified_similarity, test_size ))

    return MAE, mean_original_similarity, mean_diversified_similarity

In [60]:
diversification_factors = list(np.linspace(0,1,11))
test_size = 100
min_ratings_per_user = 30


MAEs = {}
mean_similarities_non_diversified = {}
mean_similarities_diversified = {}
for diversification_factor in diversification_factors:
    print('start testing with diversification factor equal to {}'.format(diversification_factor))
    start = time.time()
    
    MAE, mean_similarity_non_diversified, mean_similarity_diversified = perform_test(inverted_train_dict, 
                                                                                test_dict,  
                                                                                movies_metadata, 
                                                                                clique_size=200, 
                                                                                test_size = test_size, 
                                                                                min_ratings_per_user=min_ratings_per_user,
                                                                                differentiation_factor = diversification_factor)
    MAEs[diversification_factor] = MAE
    mean_similarities_non_diversified[diversification_factor] = mean_similarity_non_diversified
    mean_similarities_diversified[diversification_factor] = mean_similarity_diversified
    
    end = time.time()
    print('Elapsed time is {} seconds'.format(end-start))

isting one

clique already computed, use existing one

clique already computed, use existing one

clique already computed, use existing one

clique already computed, use existing one

clique already computed, use existing one

clique already computed, use existing one

clique already computed, use existing one

clique already computed, use existing one

clique already computed, use existing one

clique already computed, use existing one

clique already computed, use existing one

clique already computed, use existing one

clique already computed, use existing one

clique already computed, use existing one

clique already computed, use existing one

clique already computed, use existing one

clique already computed, use existing one

clique already computed, use existing one

clique already computed, use existing one

clique already computed, use existing one

clique already computed, use existing one

clique already computed, use existing one

clique already computed, use existing one


In [61]:
print('\n======= MEAN ABSOLUTE ERRORS =======')
print(MAEs)

print('\n======= MEAN SIMILARITIES NON DIVERSIFIED =======')
print(mean_similarities_non_diversified)

print('\n======= MEAN SIMILARITIES DIVERSIFIED =======')
print(mean_similarities_diversified)


======= MEAN ABSOLUTE ERRORS =======
{0.0: 3.394857027840442, 0.1: 3.4103644338427683, 0.8: 4.121056917977574, 0.30000000000000004: 3.5496776082843757, 1.0: 4.6712206852577, 0.9: 4.297544340491654, 0.2: 3.4244137739480363, 0.6000000000000001: 3.8301594960644803, 0.7000000000000001: 3.9683523746879827, 0.4: 3.6245468063691275, 0.5: 3.741854709923607}

======= MEAN SIMILARITIES NON DIVERSIFIED =======
{0.0: 14.170641125541126, 0.1: 14.170641125541126, 0.8: 14.170641125541126, 0.30000000000000004: 14.170641125541126, 1.0: 14.170641125541126, 0.9: 14.170641125541126, 0.2: 14.170641125541126, 0.6000000000000001: 14.170641125541126, 0.7000000000000001: 14.170641125541126, 0.4: 14.170641125541126, 0.5: 14.170641125541126}

======= MEAN SIMILARITIES DIVERSIFIED =======
{0.0: 14.170641125541126, 0.1: 13.68846553030303, 0.8: 8.744408832833832, 0.30000000000000004: 11.682619696969693, 1.0: 8.079911600899099, 0.9: 8.341280344655338, 0.2: 12.702509577922076, 0.6000000000000001: 9.573520775058274, 

In [62]:
scores_path = os.path.join(os.getcwd(),'scores')
item_based_scores_path = os.path.join(scores_path, 'item_based')

MAEs_path = os.path.join(item_based_scores_path, 'MAEs.txt')
sim_non_diversified_path = os.path.join(item_based_scores_path, 'intra-similarities_scores_non_diversified.txt')
sim_diversified_path = os.path.join(item_based_scores_path, 'intra-similarities_scores_diversified.txt')

with open(MAEs_path,'w') as fw:
    json.dump(MAEs, fw)

with open(sim_non_diversified_path,'w') as fw:
    json.dump(mean_similarities_non_diversified, fw)

with open(sim_diversified_path,'w') as fw:
    json.dump(mean_similarities_diversified, fw)

In [63]:
new_cliques = {}
items_keys = list(cliques.keys())
for item in items_keys:
    new_list = []
    for sub_array in cliques[item]:
        new_list.append(list(sub_array))
        
     
    new_cliques[item] = new_list
    

In [64]:
with open('./cliques_items.txt','w') as fw:
    json.dump(new_cliques, fw)

In [65]:
len(list(new_cliques.keys()))

4903